In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import scipy as sp
# import contextily as cx

import torch
import pygsp
import optuna
import joblib
import gc
import argparse
import os
import matplotlib
import pickle

from matplotlib.ticker import ScalarFormatter, StrMethodFormatter, FormatStrFormatter, FuncFormatter
from matplotlib.animation import FuncAnimation

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from optuna.samplers import TPESampler, BruteForceSampler
from torch.nn import Linear
from torch_geometric.nn.models import GraphUNet
from torch_geometric.nn import GCNConv, Sequential
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx, grid
from torchvision import datasets, transforms

from pyprojroot import here
ROOT_DIR = str(here())

matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'font.family': 'Times New Roman'})

# Function definitions
%run utils.ipynb

## Graph U-net

In [ ]:
%run utils.ipynb

# SELECT EXPERIMENT TYPE HERE
experiment_type = 'random' # 'random' or 'synthetic'

def test_model(model, n_epochs, learning_rate, penalty_rate, graph_size):

    torch.manual_seed(0)
    loss_function = torch.nn.MSELoss() 
    optimizer = torch.optim.Adam(model.parameters(),
                                lr = learning_rate,
                                weight_decay = penalty_rate)
    
    scaler = StandardScaler()
    auc = []

    test_seed_offset = 1001  #offset for picking different seeds from hyperparameter tuning

    for test_seed in range(50):

        seed = test_seed_offset + test_seed

        print(f'seed:{seed}')
        np.random.seed(seed)

        # Random graph
        G = pygsp.graphs.Sensor(graph_size, seed=seed, n_try=200)
        G.coords = G.coords*100

        # # Synthetic graph
        # G = generate_synthetic_graph(N=graph_size)

        edge_index = torch.tensor(np.array(np.nonzero(G.A.toarray())), dtype=torch.long)
        data, label = generate_data(G, size=20, anomaly_type=2)

        data = scaler.fit_transform(data)
        label_vector = label.reshape((-1,), order='F')

        error = []
        
        for snap in range(data.shape[1]):

            x = torch.Tensor(data[:,snap]).reshape(-1,1)

            model.reset_parameters()

            epochs = n_epochs
            outputs = []
            losses = []
            for epoch in range(epochs):
                        
                reconstructed = model(x, edge_index)     # Output of Autoencoder
                loss = loss_function(reconstructed, x)    # Calculating the loss function
                
                optimizer.zero_grad() # The gradients are set to zero,
                loss.backward() # the gradient is computed and stored.
                optimizer.step() # .step() performs parameter update

                # Storing the losses in a list for plotting
                losses.append(loss)
                outputs.append((epoch, x, reconstructed))

            error_snap = np.abs(reconstructed.detach() - x).numpy().flatten()
            error.extend(error_snap)
            
        error = np.array(error).reshape((-1,))
        tpr, fpr, thr = roc_params(error, label_vector, interp=True)
        auc.append(compute_auc(tpr,fpr))

    return auc


auc_list = []
for graph_size in [5, 10, 25, 50, 100, 150, 200]:
    print(f'Graph size:{graph_size}')

    best_params = joblib.load(f"../outputs/optuna_gunet_{experiment_type}/optimization_logs_{graph_size}.pkl").best_params

    n_epochs = best_params['n_epochs']
    learning_rate = best_params['learning_rate']
    penalty_rate = best_params['penalty_rate']
    hidden_channels = best_params['hidden_channels']
    depth = best_params['depth']
    pool_ratios = best_params['pool_ratios']

    model = GraphUNet(1, hidden_channels, 1, depth, pool_ratios)

    auc_list.append(test_model(model, n_epochs, learning_rate, penalty_rate, graph_size))
    

log_dir = ROOT_DIR + '/outputs/testing/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir,exist_ok=True)

log_file = log_dir + f'auc_results_gunet_{experiment_type}.pkl'
joblib.dump(auc_list, log_file)

## Weighted spectral energy

In [ ]:
%run utils.ipynb

# SELECT EXPERIMENT TYPE HERE
experiment_type = 'random' # 'random' or 'synthetic'

def test_model(cut, decay, graph_size):
    
    scaler = StandardScaler()
    auc = []

    test_seed_offset = 1001  #offset for picking different seeds from hyperparameter tuning

    for test_seed in range(50):

        seed = test_seed_offset + test_seed

        print(f'seed:{seed}')
        np.random.seed(seed)

        # # Random graph
        # G = pygsp.graphs.Sensor(graph_size, seed=seed, n_try=200)
        # G.coords = G.coords*100

        # Synthetic graph
        G = generate_synthetic_graph(N=graph_size)
        
        
        nodes = pd.DataFrame({'easting':G.coords[:,0], 'northing':G.coords[:,1]})

        data, label = generate_data(G, size=20, anomaly_type=2)
        data = scaler.fit_transform(data)

        label_vector = label.reshape((-1,), order='F')

        error = []
        
        for snap in range(data.shape[1]):

            wse = compute_wse(nodes, x=data[:,snap].reshape((-1,1)), decay=decay, cut=cut)

            error.extend(wse)
            
        error = np.array(error).reshape((-1,))
        tpr, fpr, thr = roc_params(error, label_vector, interp=True)
        auc.append(compute_auc(tpr,fpr))

    return auc


auc_list = []
for graph_size in [5, 10, 25, 50, 100, 150, 200]:
    print(f'Graph size:{graph_size}')

    best_params = joblib.load(f"../outputs/optuna_wse_{experiment_type}/optimization_logs_{graph_size}.pkl").best_params

    cut = best_params['cut']
    decay = best_params['decay']

    auc_list.append(test_model(cut, decay, graph_size))
    

log_dir = ROOT_DIR + '/outputs/testing/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir,exist_ok=True)

log_file = log_dir + f'auc_results_wse_{experiment_type}.pkl'
joblib.dump(auc_list, log_file)

## Graph filter

In [ ]:
%run utils.ipynb

# SELECT EXPERIMENT TYPE HERE
experiment_type = 'random' # 'random' or 'synthetic'

def test_model(cut, graph_size):
    
    scaler = StandardScaler()
    auc = []

    test_seed_offset = 1001  #offset for picking different seeds from hyperparameter tuning

    for test_seed in range(50):

        seed = test_seed_offset + test_seed

        print(f'seed:{seed}')
        np.random.seed(seed)

        # Random graph
        G = pygsp.graphs.Sensor(graph_size, seed=seed, n_try=200)
        G.coords = G.coords*100

        # # Synthetic graph
        # G = generate_synthetic_graph(N=graph_size)
        
        data, label = generate_data(G, size=20, anomaly_type=2)
        data = scaler.fit_transform(data)

        label_vector = label.reshape((-1,), order='F')

        Hh = hfilter(G, cut)

        error = []
        
        for snap in range(data.shape[1]):

            error.extend(np.abs(Hh @ data[:,snap]))
            
        error = np.array(error).reshape((-1,))
        tpr, fpr, thr = roc_params(error, label_vector, interp=True)
        auc.append(compute_auc(tpr,fpr))

    return auc


auc_list = []
for graph_size in [5, 10, 25, 50, 100, 150, 200]:
    print(f'Graph size:{graph_size}')

    best_params = joblib.load(f"../outputs/optuna_gfilter_{experiment_type}/optimization_logs_{graph_size}.pkl").best_params

    cut = best_params['cut']

    auc_list.append(test_model(cut, graph_size))
    

log_dir = ROOT_DIR + '/outputs/testing/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir,exist_ok=True)

log_file = log_dir + f'auc_results_gfilter_{experiment_type}.pkl'
joblib.dump(auc_list, log_file)

## Reading results

In [ ]:
methods = ['gunet_random', 'wse_random', 'gfilter_random', 'gunet_synthetic', 'wse_synthetic','gfilter_synthetic']

for method in methods:
    auc_list = joblib.load(f'../outputs/testing/auc_results_{method}.pkl')
    print(np.mean(auc_list,axis=1))